In [ ]:
import numpy as np
import random
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# --- Genetic Algorithm Setup ---
POP_SIZE = 20      # number of individuals
N_GENERATIONS = 10 # iterations
MUTATION_RATE = 0.2

# Chromosome: [max_depth, min_samples_split]
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]

def fitness(chromosome):
    max_depth, min_samples_split = chromosome
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_split=min_samples_split)
    scores = cross_val_score(model, X, y, cv=5)
    return scores.mean()

def selection(population, fitnesses):
    idx = np.argsort(fitnesses)[-2:]  # select best two
    return [population[idx[0]], population[idx[1]]]

def crossover(parent1, parent2):
    point = random.randint(0, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(chromosome):
    if random.random() < MUTATION_RATE:
        chromosome[0] = random.randint(1, 20)
    if random.random() < MUTATION_RATE:
        chromosome[1] = random.randint(2, 10)
    return chromosome

# --- Run GA ---
population = [create_chromosome() for _ in range(POP_SIZE)]

for gen in range(N_GENERATIONS):
    fitnesses = [fitness(chromo) for chromo in population]
    print(f"Generation {gen} - Best Fitness: {max(fitnesses):.4f}")

    new_population = []
    parents = selection(population, fitnesses)
    for _ in range(POP_SIZE // 2):
        child1, child2 = crossover(parents[0], parents[1])
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))

    population = new_population

# Best result
fitnesses = [fitness(chromo) for chromo in population]
best_idx = np.argmax(fitnesses)
print("Best Hyperparameters:", population[best_idx])
print("Best Accuracy:", fitnesses[best_idx])


Generation 0 - Best Fitness: 0.9733
Generation 1 - Best Fitness: 0.9733
Generation 2 - Best Fitness: 0.9733
Generation 3 - Best Fitness: 0.9733
Generation 4 - Best Fitness: 0.9733
Generation 5 - Best Fitness: 0.9733
Generation 6 - Best Fitness: 0.9733
Generation 7 - Best Fitness: 0.9733
Generation 8 - Best Fitness: 0.9733
Generation 9 - Best Fitness: 0.9733
Best Hyperparameters: [3, 4]
Best Accuracy: 0.9733333333333334


# Assignment 4 — Genetic Algorithm for Decision Tree Optimization

> Detailed explanation file

---

## Table of contents

1. Conceptual background

   * Genetic Algorithms (in-depth)
   * Representation and encodings
   * Selection, crossover, mutation (theory and options)
   * Convergence, diversity, and common pitfalls
   * GA for hyperparameter optimization (why it can work)
2. Decision Tree hyperparameters: meaning and effects

   * `max_depth`
   * `min_samples_split`
   * How these affect bias–variance and overfitting
3. How this specific code implements those concepts

   * Line-by-line walkthrough (high-level)
   * Design choices in the code and their consequences
4. Interpreting the printed results

   * What the generation best fitness shows
   * Why accuracy can fluctuate between generations
   * What the final best hyperparameters mean in practice
5. Practical considerations, improvements and experiments to try

   * Metrics, cross-validation, and evaluation protocol
   * Selection, crossover, mutation variants
   * Elitism, population diversity, and stopping criteria
   * Parallelism and computational cost
   * Reproducibility and seeding
6. Pseudocode and recommended parameter ranges
7. Common mistakes and debugging tips
8. Suggested further reading and next steps

---

# 1. Conceptual background

## Genetic Algorithms — an in-depth view

A Genetic Algorithm (GA) is a class of **stochastic optimisation algorithms** inspired by biological evolution: populations of candidate solutions (individuals) evolve over generations under the influence of selection, recombination (crossover) and mutation. The GA is a **metaheuristic** — it does not require gradient information, so it's suitable for non-differentiable, discrete, or noisy objective landscapes.

Important properties and ideas:

* **Population-based search**: instead of a single candidate (as in hill-climbing), GAs keep multiple candidates which allows parallel exploration of different regions of the search space.

* **Representation**: a candidate solution is encoded as a chromosome. Common encodings: binary strings, integer vectors, real-valued vectors, trees (GP), or permutations. The encoding determines how crossover/mutation operate.

* **Fitness function**: maps a chromosome to a scalar score (higher is better). This guides selection. Fitness often equals validation metric in ML hyperparameter tuning.

* **Selection pressure**: how strongly the algorithm favours better individuals. High selection pressure accelerates convergence but risks premature convergence (losing diversity). Low selection pressure keeps diversity but slows progress.

* **Exploration vs. exploitation**: Crossover and selection drive exploitation of good regions; mutation injects novelty (exploration). Balancing these is central to GA design.

* **Schemes and the Building Block Hypothesis**: the idea (Holland) that short, low-order, highly fit schemata (building blocks) combine through crossover to form better solutions. This remains an influential but not definitive theoretical framing.

* **Schema theorem (informal)**: describes how the expected number of copies of a schema changes generation-to-generation under selection, crossover and mutation. It provides intuition for how building blocks propagate.

* **Stochasticity and robustness**: Because GAs are stochastic, repeated runs will give slightly different results; statistical evaluation over multiple runs is recommended.

## Representation and encodings

The chosen chromosome encoding determines allowable genetic operators and the search geometry.

* **Integer vector encoding** (used in the code): each gene is an integer — convenient for integer hyperparameters like `max_depth`.

* **Real-valued encoding**: used for continuous hyperparameters; crossover might be arithmetic (weighted average).

* **Binary encoding**: classical, often used historically but less direct for integer ranges.

Design consideration: ensure encoding is compact and meaningful (adjacent values ideally have similar fitness), otherwise crossover/mutation might produce unhelpful offspring.

## Selection, crossover, mutation — theory and options

### Selection methods (popular choices)

* **Roulette-wheel (fitness-proportionate)**: probability proportional to fitness. Pros: simple. Cons: sensitive to scaling, can suffer if one individual dominates.

* **Rank selection**: individuals ranked by fitness; selection probability based on rank. Reduces sensitivity to raw fitness scale.

* **Tournament selection**: randomly pick `k` individuals, the best wins. Very popular: simple, efficient, tunable via `k`.

* **Deterministic/stochastic elitism**: keep top `e` individuals unchanged to next generation to preserve best solutions.

### Crossover operators

* **Single-point crossover**: split at one point and swap tails (this code uses an index-based split).

* **Two-point crossover**: choose two cut points and swap the middle section.

* **Uniform crossover**: for each gene, randomly choose parent.

* **Arithmetic crossover**: for real genes, take weighted average.

Choice matters: single-point crossover assumes some locality of gene interactions (i.e., neighbouring genes compose useful building blocks).

### Mutation operators

* **Random resetting**: replace gene with a random valid value (as in code).

* **Creep mutation**: for integers, add or subtract a small step; often better for local fine-tuning.

* **Gaussian perturbation**: for reals, add Gaussian noise.

Mutation rate tuning impacts exploration—too high and the search becomes random; too low and the population may stagnate.

## Convergence, diversity, and common pitfalls

* **Premature convergence**: population loses diversity and gets stuck in a local optimum.
* **Genetic drift**: random fluctuations can cause loss of alleles even without selection pressure.
* **Brittle encodings**: if small gene changes cause large fitness swings, GA will struggle.
* **Fitness evaluation cost**: if fitness is expensive (e.g., deep learning model training), GA can be too slow without parallelization.

Countermeasures: elitism, maintaining larger populations, adaptive mutation rates, niching, fitness sharing, or hybrid algorithms (GA + local search).

## Why GAs for hyperparameter tuning?

* GAs don't require gradients and can handle mixed discrete/continuous spaces.
* They explore multiple modes simultaneously and can find good combinations when hyperparameters interact non-linearly.
* However, GAs are heuristic; for many ML tasks Bayesian Optimization or Tree-structured Parzen Estimator (TPE) can be more sample-efficient. GAs shine when the fitness landscape is rugged and you can evaluate many candidates (parallel resources available).

# 2. Decision Tree hyperparameters: meaning and effects

This assignment optimises two hyperparameters of `sklearn.tree.DecisionTreeClassifier`:

* `max_depth` — maximum depth of the tree (integer >= 1 or `None`).

  * Small values bias the model toward underfitting (high bias, low variance).
  * Large values reduce bias but increase variance (risk of overfitting to training data).

* `min_samples_split` — minimum number of samples required to split an internal node (integer >= 2 or float fraction).

  * Larger minimums force the tree to make splits only on larger subsets, simplifying the tree and reducing overfitting.
  * Very small values allow deep, complex trees (overfit).

Decision trees are prone to overfitting; these hyperparameters control complexity and generalization.

**Interaction**: `max_depth` and `min_samples_split` interact: a shallow tree with low `min_samples_split` won't grow many nodes; a deep tree with high `min_samples_split` may still be constrained. GA can find non-intuitive sweet spots that balance both.

# 3. How this specific code implements those concepts

> **Note**: this section explains the implementation decisions and the algorithm flow used in the provided code.

### High-level flow

1. Load Iris dataset (`X, y`).
2. Initialise a population of `POP_SIZE` chromosomes, each `[max_depth, min_samples_split]`.
3. Evaluate fitness (5-fold CV accuracy) for each chromosome.
4. Select the top two individuals as parents.
5. Repeatedly crossover those two parents to fill the new population, applying mutation to offspring.
6. Repeat for `N_GENERATIONS`.
7. Print the best hyperparameters from the final population.

### Specific design choices and consequences

* **Population size (20)**: small-to-moderate; keeps evaluations manageable. Larger populations improve exploration but cost more evaluations.

* **Generations (10)**: a small number enough for demonstration but often insufficient to fully converge on hard problems.

* **Fitness = CV mean accuracy**: robust compared to single train/test split but increases computation (5x model fits per fitness evaluation).

* **Selection uses the top-2 (`np.argsort(fitnesses)[-2:]`)**: purely elitist selection; only the two best parents produce all children. Consequences:

  * **Very high selection pressure**: fast exploitation of the top two individuals.
  * **Diversity loss**: because only two parents contribute, population diversity collapses quickly. This accelerates convergence but increases the risk of getting trapped in a local optimum.

* **Crossover: single-point with random cut**: simple and fine for two-gene chromosomes — in fact with only two genes, the crossover point will be 0 or 1, so children are either copies or swapped genes. That means crossover is effectively a swap of whole genes. With two genes this is fine, but there is limited recombination granularity.

* **Mutation: random resetting with independent probability per gene**: introduces novelty but with the given `MUTATION_RATE=0.2` and only two genes, the expected number of mutated genes per offspring is `2 * 0.2 = 0.4`.

* **No explicit elitism in the new population**: although selection uses the top two parents to create children, none of the parents are carried over unchanged unless crossover + mutation recreate them. This risks losing high-fitness individuals by chance.

# 4. Interpreting the printed results

### `Generation g - Best Fitness: 0.xxx`

* For each generation the code prints the maximum fitness observed across the population (i.e., best mean cross-validation accuracy).
* If the GA is improving, you should see a non-decreasing trend in the best fitness across generations (often increasing, but stochastic fluctuations can occur).

### Why accuracy can fluctuate or plateau

* **Stochastic operators** (mutation and crossover) — the best individual might be replaced by a slightly worse offspring.
* **Discrete gene space and small population** — jumps can be abrupt and not smooth.
* **Cross-validation variance** — the CV mean itself has variance; small changes in hyperparameters may not yield reliably different CV means.

### Final best hyperparameters and Best Accuracy

* These are drawn from the final population only. Because the algorithm doesn’t keep a global archive of best individuals across all generations, the best ever found might have existed in a previous generation and then been lost. To reliably keep the global best, one should implement **elitism** (always copy the best-so-far into the next generation).

* The reported `Best Accuracy` is the CV mean on the last-population individual — treat it as an estimate, not the absolute optimal configuration. It’s good practice to retrain a final model with those hyperparameters on the full training set and evaluate on an independent test set.

# 5. Practical considerations, improvements and experiments to try

This section lists practical changes that will typically improve GA performance and measurement reliability.

## Evaluation and metrics

* **Use a hold-out test set**: After GA selects hyperparameters, retrain on the full training set and evaluate on a test set not used during GA to estimate true generalization.
* **Average over multiple GA runs**: Because GAs are stochastic, run the whole GA multiple times and aggregate results (mean and std of best fitness).
* **Consider other metrics**: accuracy can be insufficient for imbalanced classes; use F1, ROC-AUC, etc.

## Selection improvements

* **Tournament selection (k=3 or 5)**: preserves diversity and gives controllable selection pressure.
* **Roulette (fitness-proportionate)**: use with scaled fitness when needed.
* **Rank selection**: robust to scaling and outliers.

## Maintain elite individuals

* **Elitism**: copy the top `e` individuals into the next generation unchanged — avoids losing the best solution.

## Crossover and mutation variants

* With more genes, try **two-point** or **uniform crossover**.
* Replace random reset mutation with **creep mutation** (add/subtract a small integer) to fine-tune continuous-like integer hyperparameters.
* Consider **adaptive mutation**: start with higher mutation rate and reduce over generations.

## Population and generation strategy

* **Larger population** increases exploration but costs more evaluations.
* **More generations** allow further refinement.
* **Steady-state GA**: replace only a few individuals per generation instead of whole-population replacement. This often preserves diversity and smooths progress.

## Hybrid approaches

* **Memetic algorithms**: combine GA with local search (e.g., perform small local search around each child to fine-tune).
* **Bayesian Optimization + GA**: use GA to explore broadly then BO to refine promising regions.

## Parallelism and computational cost

Fitness evaluation (cross-validation) is embarrassingly parallel — evaluate multiple individuals in parallel (e.g., with `joblib` or multiprocessing). This is essential when fitness is expensive.

## Reproducibility

* Seed random number generators (`random.seed`, `np.random.seed`) for reproducibility during debugging.
* Log hyperparameter trials and CV scores.

# 6. Pseudocode and recommended parameter ranges

### Pseudocode (improved version with elitism and tournament selection)

```text
initialize population P
evaluate fitness for each individual in P
for gen in 1..G:
    select parents using tournament selection
    create offspring via crossover
    mutate offspring
    evaluate offspring fitness
    form new population by: keep top E elites from P, then add best offspring until population full
    optionally update global_best
return global_best
```

### Recommended parameter ranges (starting points)

* `POP_SIZE`: 20–200 (start with 20–50 for small problems)
* `N_GENERATIONS`: 10–200 (depends on budget)
* `MUTATION_RATE`: 0.01–0.3 — lower if genes are continuous/fine-grained, higher for rugged landscapes
* `Tournament k`: 2–5
* `Elitism E`: 1–5 individuals

# 7. Common mistakes and debugging tips

* **Not using a hold-out test set**: leads to over-optimistic claims.
* **Relying on single GA run**: risks reporting lucky results.
* **Having only two parents generate all children**: reduces diversity, often a bad design for more complex problems.
* **Using expensive fitness without parallelism**: leads to very slow tuning.
* **Not keeping the global best**: lose good solutions across generations; add elitism or track best-so-far separately.

Debugging tips:

* Visualize best, median and worst fitness per generation (boxplots or line plots).
* Track population variety (e.g., histogram of genes) to detect premature convergence.
* Temporarily disable mutation or crossover to isolate issues.

# 8. Suggested further reading and next steps

* David E. Goldberg — *Genetic Algorithms in Search, Optimization and Machine Learning* (classic)
* Melanie Mitchell — *An Introduction to Genetic Algorithms* (accessible overview)
* Papers and tutorials on evolutionary algorithms for hyperparameter optimization
* Explore `DEAP` (Distributed Evolutionary Algorithms in Python) — a mature GA toolkit that supports many operators and parallel evaluation.

---

## Appendix: quick checklist to improve the provided script

* [ ] Add `random.seed(...)` and `np.random.seed(...)` for reproducibility when testing.
* [ ] Replace `selection` with tournament selection for better diversity.
* [ ] Add elitism: copy the best `e` individuals unchanged into `new_population`.
* [ ] Record and keep `global_best` across generations.
* [ ] Optionally replace random reset mutation with `±1` creep for `max_depth`.
* [ ] Evaluate the final recommended hyperparameters on a held-out test set and present a confusion matrix.
* [ ] Run the GA multiple times and report mean/std of best accuracy.

---

*End of file.*
